In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
music = pd.read_csv('songDb.tsv', sep='\t', encoding='ISO-8859-1')

c:\Users\shush\Anaconda3\envs\project\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
music.head()

,Name,Danceability,Energy,Key,Loudness,Mode,Speechness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Type,ID,Uri,Ref_Track,URL_features,Duration_ms,time_signature,Genre
0,YuveYuveYu,0.624,0.857,10.0,-6.250,0.0,0.0542,0.020800,0.206000,0.110,0.324,131.926,audio_features,6J2VvzKwWc2f0JP5RQVZjq,spotify:track:6J2VvzKwWc2f0JP5RQVZjq,https://api.spotify.com/v1/tracks/6J2VvzKwWc2f...,https://api.spotify.com/v1/audio-analysis/6J2V...,282920.0,4.0,celticmetal
1,Gloryhammer,0.517,0.916,0.0,-4.933,1.0,0.0559,0.000182,0.001910,0.306,0.444,135.996,audio_features,4HA34COgxgVJ6zK88UN4Ik,spotify:track:4HA34COgxgVJ6zK88UN4Ik,https://api.spotify.com/v1/tracks/4HA34COgxgVJ...,https://api.spotify.com/v1/audio-analysis/4HA3...,300320.0,4.0,celticmetal
2,Nostos,0.251,0.894,8.0,-4.103,0.0,0.0570,0.014400,0.000000,0.123,0.297,114.223,audio_features,3W6Xik6Xxf06JuUoZSATlD,spotify:track:3W6Xik6Xxf06JuUoZSATlD,https://api.spotify.com/v1/tracks/3W6Xik6Xxf06...,https://api.spotify.com/v1/audio-analysis/3W6X...,175353.0,4.0,celticmetal
3,Yggdrasil,0.469,0.743,1.0,-5.570,0.0,0.0272,0.002220,0.000111,0.276,0.481,86.953,audio_features,2gGveBaLJQMtJ43X4UL5kH,spotify:track:2gGveBaLJQMtJ43X4UL5kH,https://api.spotify.com/v1/tracks/2gGveBaLJQMt...,https://api.spotify.com/v1/audio-analysis/2gGv...,272292.0,4.0,celticmetal
4,Incense&Iron,0.487,0.952,1.0,-4.429,0.0,0.0613,0.000228,0.000000,0.161,0.329,125.993,audio_features,1lRF81A1C9QoCgBcEop2zg,spotify:track:1lRF81A1C9QoCgBcEop2zg,https://api.spotify.com/v1/tracks/1lRF81A1C9Qo...,https://api.spotify.com/v1/audio-analysis/1lRF...,237933.0,4.0,celticmetal


In [4]:
df = music # for testing

df = df.rename({'Genre':'genres'}, axis='columns') # rename to match 
df = df.dropna()

# combines subgenres into more general Genres. This function removes all genres outside of the top 5.
# You can alter this to include more than the top 5 genres
def CombineGenre(str_in):

    try:
        str_in = str_in.lower()
        str_in = str_in.strip()
        
        if 'rock' in str_in or 'prog' in str_in or 'gaze' in str_in or 'psych' in str_in:
            output = 'Rock'
        elif 'hip hop' in str_in or 'rap' in str_in or 'grime' in str_in or 'trap' in str_in:
            output = 'Rap'
        elif 'pop' in str_in:
            output = 'Pop'
        elif ('edm' in str_in or 'electronic' in str_in or 'house' in str_in or 'industrial' in str_in or 'glitch' in str_in 
                or 'idm' in str_in or 'techno' in str_in or 'garage' in str_in or 'reggeaton' in str_in or 'synth' in str_in 
                or 'dubstep' in str_in or 'trance' in str_in or 'wave' in str_in or 'electro' in str_in):
            output = 'Electronic'
        elif 'indie' in str_in:
            output = 'Indie'
        else:
            output  = np.nan
    except:
        output = np.nan
        
    return output

# applys above function to combine genres
df['genres'] = df['genres'].apply(CombineGenre)
df = df.dropna() # removes all genres that were not combined before 
df = df.reset_index() # resets the index since we removed many rows
df # shows now filtered df of music
# df['genres'].value_counts() # shows number of songs in each top 5 genres

,index,Name,Danceability,Energy,Key,Loudness,Mode,Speechness,Acousticness,Instrumentalness,...,Valence,Tempo,Type,ID,Uri,Ref_Track,URL_features,Duration_ms,time_signature,genres
0,802,Milion,0.734,0.677,0.0,-4.120,0.0,0.0471,0.087000,0.000000,...,0.189,75.053,audio_features,7lDuA0zGpNONIAsrwWr4yr,spotify:track:7lDuA0zGpNONIAsrwWr4yr,https://api.spotify.com/v1/tracks/7lDuA0zGpNON...,https://api.spotify.com/v1/audio-analysis/7lDu...,158500.0,4.0,Rap
1,803,Ludanoc,0.713,0.878,1.0,-4.104,1.0,0.0513,0.016300,0.000000,...,0.725,130.039,audio_features,6BpEsWk5MjSz3s45dg9RNS,spotify:track:6BpEsWk5MjSz3s45dg9RNS,https://api.spotify.com/v1/tracks/6BpEsWk5MjSz...,https://api.spotify.com/v1/audio-analysis/6BpE...,194400.0,4.0,Rap
2,804,Milion,0.715,0.913,1.0,-4.337,0.0,0.0602,0.006860,0.000000,...,0.669,97.97,audio_features,4XNdzdcmGM8UrrJV8C1k4a,spotify:track:4XNdzdcmGM8UrrJV8C1k4a,https://api.spotify.com/v1/tracks/4XNdzdcmGM8U...,https://api.spotify.com/v1/audio-analysis/4XNd...,179999.0,4.0,Rap
3,805,12sati,0.844,0.856,6.0,-5.193,0.0,0.0453,0.172000,0.000016,...,0.947,129.994,audio_features,0QZj1doaH1iyj8ubJ9wm8F,spotify:track:0QZj1doaH1iyj8ubJ9wm8F,https://api.spotify.com/v1/tracks/0QZj1doaH1iy...,https://api.spotify.com/v1/audio-analysis/0QZj...,180923.0,4.0,Rap
4,806,PetaBrzina,0.625,0.939,7.0,-3.185,0.0,0.1350,0.270000,0.000007,...,0.794,172.016,audio_features,1RhlloW4u7N2Y4h2EOgL7q,spotify:track:1RhlloW4u7N2Y4h2EOgL7q,https://api.spotify.com/v1/tracks/1RhlloW4u7N2...,https://api.spotify.com/v1/audio-analysis/1Rhl...,210000.0,4.0,Rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51957,131575,Youth,0.568,0.708,8.0,-9.960,1.0,0.0601,0.007930,0.000528,...,0.214,127.741,audio_features,5AozgGtATNJi2Yx5Vb2InS,spotify:track:5AozgGtATNJi2Yx5Vb2InS,https://api.spotify.com/v1/tracks/5AozgGtATNJi...,https://api.spotify.com/v1/audio-analysis/5Aoz...,259560.0,4,Rock
51958,131576,IFoundOut,0.470,0.909,4.0,-1.674,1.0,0.0546,0.061100,0.000000,...,0.607,146.986,audio_features,34XDIqYypZc3jyGRDgd5p4,spotify:track:34XDIqYypZc3jyGRDgd5p4,https://api.spotify.com/v1/tracks/34XDIqYypZc3...,https://api.spotify.com/v1/audio-analysis/34XD...,127400.0,4,Rock
51959,131577,Animal,0.272,0.918,11.0,-2.589,0.0,0.0625,0.000749,0.009200,...,0.530,139.574,audio_features,5MpD4w1JTHkesmjn9I8Qo5,spotify:track:5MpD4w1JTHkesmjn9I8Qo5,https://api.spotify.com/v1/tracks/5MpD4w1JTHke...,https://api.spotify.com/v1/audio-analysis/5MpD...,159627.0,4,Rock
51960,131578,PostBreak-UpSex,0.402,0.902,5.0,-4.115,1.0,0.0469,0.000073,0.004650,...,0.569,136.883,audio_features,77GZxme1GMNbbooEH8nHNX,spotify:track:77GZxme1GMNbbooEH8nHNX,https://api.spotify.com/v1/tracks/77GZxme1GMNb...,https://api.spotify.com/v1/audio-analysis/77GZ...,174453.0,4,Rock


In [5]:
feature_cols = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
                'Speechness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence',
                'Tempo', 'Duration_ms', 'time_signature']

df[feature_cols] = df[feature_cols].astype(np.float64)

In [6]:
# Start by removing all columns that are not features 

feature_cols = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
                'Speechness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence',
                'Tempo', 'Duration_ms', 'time_signature']
target_col = 'genres'

X = df[feature_cols]
y = df[target_col]

print(f'Feature cols: {feature_cols}')
print(f'Target: {target_col}')

Feature cols: ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'time_signature']
Target: genres


In [7]:
from sklearn.model_selection import train_test_split
#Split data into train-val and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=69)

# Split train-val into train and val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)

print(f'X_train: {X_train.shape}')
print(f'X_val: {X_val.shape}')
print(f'X_test: {X_test.shape}')

X_train: (37412, 13)
X_val: (4157, 13)
X_test: (10393, 13)


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(df[target_col].unique())
#Scale and convert data to np array
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(le.transform(y_train))
X_val, y_val = np.array(X_val), np.array(le.transform(y_val))
X_test, y_test = np.array(X_test), np.array(le.transform(y_test))

In [9]:
print(y_train)

[4 0 0 ... 0 0 2]


In [10]:
class GenreDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [11]:
train_dataset = GenreDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = GenreDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = GenreDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

In [12]:
target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)

In [13]:
count = np.bincount(y_train)
# print(count)
class_weights = 1./torch.tensor(count, dtype=torch.float) 
print(class_weights)

tensor([1.3236e-04, 1.3856e-04, 9.5420e-05, 5.4945e-04, 9.6712e-05])


In [14]:
class_weights_all = class_weights[target_list]
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

In [15]:
EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 0.0007
NUM_FEATURES = len(X.columns)
NUM_CLASSES = 5

In [16]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          sampler=weighted_sampler
)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [30]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [31]:
model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

MulticlassClassification(
  (layer_1): Linear(in_features=13, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=5, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [20]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [28]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [24]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [32]:


for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0

    print("Training...")

    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()

        y_train_pred = model(X_train_batch)

        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        

    print("Validating")    
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()

    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                                
        
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')


  0%|          | 0/30 [00:00<?, ?it/s]

Training...
Validating
Epoch 001: | Train Loss: 1.06447 | Val Loss: 1.59072 | Train Acc: 38.996| Val Acc: 27.376
Training...
Validating
Epoch 002: | Train Loss: 1.01006 | Val Loss: 1.60527 | Train Acc: 41.204| Val Acc: 27.905
Training...
Validating
Epoch 003: | Train Loss: 0.99475 | Val Loss: 1.62671 | Train Acc: 42.171| Val Acc: 26.606
Training...
Validating
Epoch 004: | Train Loss: 0.98144 | Val Loss: 1.55001 | Train Acc: 42.780| Val Acc: 29.709
Training...
Validating
Epoch 005: | Train Loss: 0.96678 | Val Loss: 1.54859 | Train Acc: 42.794| Val Acc: 30.551
Training...
Validating
Epoch 006: | Train Loss: 0.95098 | Val Loss: 1.59766 | Train Acc: 43.354| Val Acc: 27.881
Training...
Validating
Epoch 007: | Train Loss: 0.95580 | Val Loss: 1.58256 | Train Acc: 43.541| Val Acc: 29.156
Training...
Validating
Epoch 008: | Train Loss: 0.95372 | Val Loss: 1.49412 | Train Acc: 43.730| Val Acc: 30.815
Training...
Validating
Epoch 009: | Train Loss: 0.95133 | Val Loss: 1.49310 | Train Acc: 43.592|